<a href="https://colab.research.google.com/github/Drishya24-lab/ProductReviewSentiment/blob/main/Product_Review_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
#Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

#Extract Data from Zip File
import zipfile
zipfile_path=('/content/drive/MyDrive/Dataset-SA.csv.zip')
with zipfile.ZipFile(zipfile_path,'r') as zip_ref:
  zip_ref.extractall('/content/drive/MyDrive')

#Load Data into Pandas DataFrame
import pandas as pd
data=pd.read_csv('/content/drive/MyDrive/Dataset-SA.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
## Display Data Information

print("**First 2 rows**")
print(data.head(2))
print('\n')
print('**DataFrame Information**')
data.info()



**First 2 rows**
                                        product_name product_price Rate  \
0  Candes 12 L Room/Personal Air Cooler??????(Whi...          3999    5   
1  Candes 12 L Room/Personal Air Cooler??????(Whi...          3999    5   

    Review                                            Summary Sentiment  
0   super!  great cooler excellent air flow and for this p...  positive  
1  awesome              best budget 2 fit cooler nice cooling  positive  


**DataFrame Information**
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205052 entries, 0 to 205051
Data columns (total 6 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   product_name   205052 non-null  object
 1   product_price  205052 non-null  object
 2   Rate           205052 non-null  object
 3   Review         180388 non-null  object
 4   Summary        205041 non-null  object
 5   Sentiment      205052 non-null  object
dtypes: object(6)
memory usage: 9.4+ MB


In [13]:
#Clean and Convert 'product_price' column

print(data['product_price'].unique())
data['product_price'] = pd.to_numeric(data['product_price'], errors='coerce')    # Remove non-numeric values in the 'product_price' column
data = data.dropna(subset=['product_price'])                                     # Drop rows with NaN values in 'product_price' after coercing to numeric
data['product_price'] = data['product_price'].astype(int)
print('\n')
print(data['product_price'].unique())

['3999' '8999' '7999' '9999' '1199' '499' '1999' '1099' '997' '1499' '435'
 '1349' '30999' '13999' '9990' '14299' '5298' '7599' '11999' '329' '425'
 '249' '302' '59' '245' '79' '349' '449' '340' '299' '469' '26990' '23479'
 '29390' '29990' '20990' '44490' '25990' '38490' '42000' '44890' '31590'
 '41990' '52990' '33990' '18990' '50999' '45550' '1401' '359' '1453' '254'
 '205' '1256' '1547' '195' '575' '366' '209' '219' '549' '859' '210' '215'
 '1142' '235' '221' '1599' '2454' '6099' '2399' '599' '849' '699'
 'pigeon favourite electric kettle15 l silver black' '4449' '4098' '5599'
 '5499' '1448' '3569' '2879' '1799' '1329' '5390' '11500' '1220' '9050'
 '6505' '6495' '11595' '7649' '4399' '6029' '6299' '5919' '6390' '2695'
 '2949' '7909' '4499' '6525' '6589' '5039' '4219' '4319' '7499' '379'
 '3499' '6999' '224' '227' '385' '199' '999' '480' '427' '236' '690' '799'
 '649' '190' '335' '285' '749' '218' '179' '197' '303' '269' '1549' '185'
 '188' '189' '665' '125' '2190' '2690' '323' '315' 

<ipython-input-13-64f7b84d8911>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['product_price'] = data['product_price'].astype(int)


In [14]:
#Clean and Convert 'Rate' column
print(data['Rate'].unique())
data['Rate'] = data['Rate'].astype(int)
print('\n')

print(data['Rate'].unique())

['5' '3' '1' '4' '2']


[5 3 1 4 2]


In [15]:
#Convert 'Sentiment' column to categorical type

data['Sentiment'] = data['Sentiment'].astype('category')



In [16]:
#Handle Missing Values

print('Null values in each column')
print(data.isnull().sum())
data=data.dropna(subset=['Review'])
data['Summary']=data['Summary'].fillna('No Summary')
print(data.isnull().sum())


Null values in each column
product_name         0
product_price        0
Rate                 0
Review           24664
Summary             11
Sentiment            0
dtype: int64
product_name     0
product_price    0
Rate             0
Review           0
Summary          0
Sentiment        0
dtype: int64


<ipython-input-16-72bfe0f5e035>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Summary']=data['Summary'].fillna('No Summary')


In [17]:
# Text Cleaning

import re

def clean_text(text):
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Remove punctuation and special characters
    return text

data['Review'] = data['Review'].apply(clean_text)


In [18]:
#Convert to Lowercase

data['Review'] = data['Review'].str.lower()


In [19]:
#Tokenization

from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')

data['Review'] = data['Review'].apply(word_tokenize)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [20]:
 #Remove Stop Words

 from nltk.corpus import stopwords
nltk.download('stopwords')

stop_words = set(stopwords.words('english'))
data['Review'] = data['Review'].apply(lambda x: [word for word in x if word not in stop_words])


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [21]:
# Lemmatization/Stemming

from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')

lemmatizer = WordNetLemmatizer()
data['Review'] = data['Review'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])


[nltk_data] Downloading package wordnet to /root/nltk_data...


In [22]:
#Reconstruct Cleaned Text

data['Review'] = data['Review'].apply(lambda x: ' '.join(x))


In [23]:
#Feature Extraction - Bag of Words
from sklearn.feature_extraction.text import CountVectorizer

data['Review'] = data['Review'].astype(str) #Convert the column to string type

# Create the BoW model
vectorizer = CountVectorizer()
X_bow = vectorizer.fit_transform(data['Review'])


In [24]:

#Feature Extraction - TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer

# Create the TF-IDF model
tfidf_vectorizer = TfidfVectorizer()
X_tfidf = tfidf_vectorizer.fit_transform(data['Review'])



In [25]:
#Encode Target Variable
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(data['Sentiment'])



In [26]:
#Split Dataset into Training and Testing Sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)


In [27]:
#Model Training - Naive Bayes
from sklearn.naive_bayes import MultinomialNB

nb_model = MultinomialNB()
nb_model.fit(X_train, y_train)
y_pred = nb_model.predict(X_test)


In [28]:
# Evaluate Naive Bayes Model
from sklearn.metrics import classification_report, accuracy_score

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.901543919948998

Classification Report:
               precision    recall  f1-score   support

           0       0.85      0.71      0.77      4894
           1       0.06      0.00      0.01      1763
           2       0.91      0.99      0.95     29420

    accuracy                           0.90     36077
   macro avg       0.61      0.57      0.58     36077
weighted avg       0.86      0.90      0.88     36077



In [29]:
#Model Training - Logistic Regression
from sklearn.linear_model import LogisticRegression

lr_model = LogisticRegression(max_iter=1000)
lr_model.fit(X_train, y_train)
y_pred_lr = lr_model.predict(X_test)
print("\nLogistic Regression Report:\n", classification_report(y_test, y_pred_lr))



Logistic Regression Report:
               precision    recall  f1-score   support

           0       0.85      0.75      0.80      4894
           1       0.00      0.00      0.00      1763
           2       0.92      0.99      0.95     29420

    accuracy                           0.91     36077
   macro avg       0.59      0.58      0.58     36077
weighted avg       0.86      0.91      0.88     36077



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [30]:
#Model Training - SVM with Grid Search

from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

param_grid = {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf']}
grid_search = GridSearchCV(SVC(), param_grid, scoring='accuracy', cv=3)
grid_search.fit(X_train, y_train)
print("Best Parameters:", grid_search.best_params_)


Best Parameters: {'C': 1, 'kernel': 'linear'}


In [31]:
#Best SVM Model Training
best_svm_model = SVC(C=1, kernel='linear')
best_svm_model.fit(X_train, y_train)
y_pred_svm = best_svm_model.predict(X_test)


In [32]:
# Evaluate SVM Model

print("SVM Model Accuracy:", accuracy_score(y_test, y_pred_svm))
print("\nClassification Report:\n", classification_report(y_test, y_pred_svm))


SVM Model Accuracy: 0.9081963577902818

Classification Report:
               precision    recall  f1-score   support

           0       0.85      0.75      0.80      4894
           1       0.00      0.00      0.00      1763
           2       0.92      0.99      0.95     29420

    accuracy                           0.91     36077
   macro avg       0.59      0.58      0.58     36077
weighted avg       0.86      0.91      0.88     36077



In [33]:
#Save the Model

import joblib
joblib.dump(best_svm_model, 'best_svm_sentiment_model.pkl')


['best_svm_sentiment_model.pkl']